In [1]:
import numpy as np
import torch
import copy
from metabci.brainda.algorithms.deep_learning import ShallowNet_Sparse
from metabci.brainda.algorithms.utils.model_selection import (
    set_random_seeds,
    generate_kfold_indices, match_kfold_indices)
from metabci.brainda.datasets import Wang2016, BETA
# 三个SSVEP Datasets：Nakanishi2015、Wang2016、BETA
from metabci.brainda.paradigms import SSVEP
from skorch.helper import predefined_split

In [2]:
#**************************************************
# Benchmark数据集读取处理
#**************************************************
Bench_dataset = Wang2016()
subject_list = list(range(1, 21))  # 被试编号从1到20
for s in subject_list:
    Bench_dataset.data_path(subject=s, path="E:\\MetaBCI-master\\mne_data")  # 依次为每个被试设置路径
events = Bench_dataset.events.keys()
freq_list = [str(Bench_dataset.get_freq(event)) for event in events]  # 获得所有刺激的频率
freq_map = {i: freq for i, freq in enumerate(freq_list)}  # 标签到频率的映射

Bench_subjects = list(range(1, 21))  # 使用S1–S20被试进行训练
Bench_paradigm = SSVEP(
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],  # 选择电极通道
    intervals=[(0.14, 1.14)],  # 分析时间窗最长-0.5-5.5s ,0.0是刺激开始
    events=freq_list,  # 选择所有刺激频率
    srate=250  # 采样率
)

# add 5-90Hz bandpass filter in raw hook
# 对原始EEG信号做5-90Hz带通滤波。
def raw_hook(raw, caches):
    raw.filter(5, 90, l_trans_bandwidth=2, h_trans_bandwidth=5, phase='zero-double')
    caches['raw_stage'] = caches.get('raw_stage', -1) + 1
    return raw, caches

Bench_paradigm.register_raw_hook(raw_hook)



# 获取微调数据
X_Bench, y_Bench, meta_Bench = Bench_paradigm.get_data(
    Bench_dataset,
    subjects=Bench_subjects,
    return_concat=True,
    n_jobs=None,
    verbose=False
)


--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun/S5.mat.7z
--------ssssss, /upload/yijun/S6.mat.7z
--------ssssss, /upload/yijun/S7.mat.7z
--------ssssss, /upload/yijun/S8.mat.7z
--------ssssss, /upload/yijun/S9.mat.7z
--------ssssss, /upload/yijun/S10.mat.7z
--------ssssss, /upload/yijun/S11.mat.7z
--------ssssss, /upload/yijun/S12.mat.7z
--------ssssss, /upload/yijun/S13.mat.7z
--------ssssss, /upload/yijun/S14.mat.7z
--------ssssss, /upload/yijun/S15.mat.7z
--------ssssss, /upload/yijun/S16.mat.7z
--------ssssss, /upload/yijun/S17.mat.7z
--------ssssss, /upload/yijun/S18.mat.7z
--------ssssss, /upload/yijun/S19.mat.7z
--------ssssss, /upload/yijun/S20.mat.7z
--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun

In [3]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 60
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0_10_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0243        4.7184       0.0350        4.7065     +  0.0100  1.1997
      2       0.0788        4.0373       0.1138        3.8868     +  0.0100  0.9455
      3       0.2360        3.1032       0.3450        2.7855     +  0.0100  0.8979
      4       0.4300        2.2863       0.4338        2.4377     +  0.0100  0.9530
      5       0.4940        1.9480       0.4587        2.3244     +  0.0100  0.9163
      6       0.5260        1.7918       0.4813        2.3912        0.0100  0.9661
      7       0.5380        1.7172       0.5100        2.2805     +  0.0100  0.8774
      8       0.5673        1.5890       0.5038        2.2741     +  0.0100  0.9252
      9       0.5667        1.5547       0.4800        2.4329        0

In [4]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 20
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0_20_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0262        7.1185       0.0387        4.8334     +  0.0100  1.0482
      2       0.0340        6.3211       0.0537        4.3170     +  0.0100  0.9436
      3       0.0558        5.8234       0.0762        4.0376     +  0.0100  0.9052
      4       0.1075        5.0399       0.2712        3.0614     +  0.0100  0.8869
      5       0.2225        4.0602       0.4200        2.2561     +  0.0100  0.9044
      6       0.3292        3.3325       0.4750        2.0614     +  0.0100  0.9562
      7       0.3670        2.9897       0.5088        1.9385     +  0.0100  0.9094
      8       0.4025        2.7267       0.4875        1.9256     +  0.0100  0.9014
      9       0.4158        2.5301       0.5375        1.8516     +  0

In [5]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 30
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0_30_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0232        6.1949       0.0413        4.4116     +  0.0100  1.0550
      2       0.0548        5.2267       0.0650        4.0537     +  0.0100  0.9369
      3       0.1165        4.4733       0.1988        3.2301     +  0.0100  0.9053
      4       0.2725        3.3572       0.4000        2.4373     +  0.0100  0.8825
      5       0.3815        2.7636       0.4900        2.0364     +  0.0100  0.8892
      6       0.4557        2.3365       0.5175        1.9896     +  0.0100  0.9361
      7       0.4888        2.1340       0.5500        1.7837     +  0.0100  0.9088
      8       0.5168        1.9956       0.5587        1.6840     +  0.0100  0.9080
      9       0.5547        1.7256       0.5813        1.6491     +  0

In [6]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 40
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0_40_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0240        5.6947       0.0300        4.4912     +  0.0100  0.8912
      2       0.0772        4.5602       0.1288        3.6761     +  0.0100  0.9149
      3       0.2195        3.4312       0.3412        2.7091     +  0.0100  0.8620
      4       0.4015        2.5259       0.4238        2.2511     +  0.0100  0.8498
      5       0.4700        2.1591       0.4550        2.1352     +  0.0100  0.8702
      6       0.4975        2.0002       0.4412        2.2699        0.0100  0.9019
      7       0.5210        1.9050       0.4675        2.2058        0.0100  0.9105
      8       0.5340        1.8172       0.4612        2.1522        0.0100  0.8460
      9       0.5455        1.7389       0.4637        2.1942        0

In [7]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 50
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0_50_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0265        5.0515       0.0312        4.6255     +  0.0100  0.9214
      2       0.0755        4.2319       0.1150        3.7796     +  0.0100  0.8741
      3       0.2375        3.1899       0.3650        2.7011     +  0.0100  0.8624
      4       0.4075        2.3949       0.4475        2.3308     +  0.0100  0.8918
      5       0.4840        2.0166       0.4637        2.1737     +  0.0100  0.9002
      6       0.5158        1.8534       0.4650        2.3445        0.0100  0.9037
      7       0.5337        1.7615       0.4788        2.2118        0.0100  0.8538
      8       0.5527        1.6724       0.4825        2.1603     +  0.0100  0.8707
      9       0.5623        1.5675       0.4950        2.2038        0

In [8]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 60
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0_60_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0243        4.7184       0.0350        4.7065     +  0.0100  0.8630
      2       0.0788        4.0373       0.1138        3.8868     +  0.0100  0.8844
      3       0.2360        3.1032       0.3450        2.7855     +  0.0100  0.9235
      4       0.4300        2.2863       0.4338        2.4377     +  0.0100  0.8479
      5       0.4940        1.9480       0.4587        2.3244     +  0.0100  0.8562
      6       0.5260        1.7918       0.4813        2.3912        0.0100  0.8668
      7       0.5380        1.7172       0.5100        2.2805     +  0.0100  0.8959
      8       0.5673        1.5890       0.5038        2.2741     +  0.0100  0.8964
      9       0.5667        1.5547       0.4800        2.4329        0

In [9]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 70
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0_70_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0272        4.5030       0.0262        4.7936     +  0.0100  0.8842
      2       0.0845        3.8633       0.1338        3.8835     +  0.0100  0.8485
      3       0.2540        2.9556       0.3700        3.0021     +  0.0100  0.8585
      4       0.4477        2.1947       0.4387        2.5556     +  0.0100  0.8832
      5       0.5022        1.8964       0.4675        2.4193     +  0.0100  0.9309
      6       0.5385        1.7429       0.4688        2.5879        0.0100  0.8435
      7       0.5515        1.6616       0.4913        2.4936        0.0100  0.8592
      8       0.5710        1.5544       0.5062        2.3939     +  0.0100  0.8744
      9       0.5750        1.5310       0.4925        2.6513        0

In [10]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 80
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0_80_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0238        4.3479       0.0350        4.7547     +  0.0100  0.8918
      2       0.0907        3.7454       0.1363        3.9465     +  0.0100  0.8597
      3       0.2730        2.8570       0.3887        3.1080     +  0.0100  0.8682
      4       0.4632        2.1243       0.4437        2.8494     +  0.0100  0.8911
      5       0.5245        1.8269       0.4863        2.5659     +  0.0100  0.9150
      6       0.5515        1.6903       0.4725        2.7595        0.0100  0.8497
      7       0.5620        1.6219       0.4838        2.7468        0.0100  0.8664
      8       0.5743        1.5395       0.4988        2.6419        0.0100  0.8920
      9       0.5825        1.4876       0.4988        2.8928        0

In [11]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 90
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0_90_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0255        4.2396       0.0425        4.8161     +  0.0100  0.8621
      2       0.0843        3.6945       0.1388        4.1214     +  0.0100  0.8899
      3       0.2647        2.9108       0.3787        3.3385     +  0.0100  0.9085
      4       0.4610        2.1457       0.4550        3.0121     +  0.0100  0.8771
      5       0.5150        1.8413       0.5025        2.6865     +  0.0100  0.8494
      6       0.5457        1.6789       0.4938        3.0008        0.0100  0.8742
      7       0.5723        1.5982       0.4938        2.8116        0.0100  0.8944
      8       0.5785        1.5217       0.5038        2.8191        0.0100  0.8994
      9       0.5927        1.4521       0.5075        2.9182        0

In [12]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 100
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0_100_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0290        4.1458       0.0338        4.9619     +  0.0100  0.9006
      2       0.0895        3.6361       0.1537        4.1337     +  0.0100  0.8924
      3       0.2963        2.7593       0.3937        3.4932     +  0.0100  0.9064
      4       0.4760        2.0852       0.4637        3.2511     +  0.0100  0.9027
      5       0.5225        1.8039       0.4925        3.1106     +  0.0100  0.8932
      6       0.5595        1.6631       0.4938        3.1748        0.0100  0.8790
      7       0.5673        1.5660       0.5062        3.0270     +  0.0100  0.8810
      8       0.5940        1.4785       0.5225        2.9979     +  0.0100  0.8940
      9       0.6015        1.4184       0.5200        3.1269        0

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0_10_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 10
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 65.4%
subject 24 的准确率: 66.7%
subject 27 的准确率: 70.0%
subject 28 的准确率: 66.2%
subject 31 的准确率: 67.1%
subject 34 的准确率: 61.7%
subject 35 的准确率: 73.3%
所有被试的准确率： {'22': 0.6541666666666667, '24': 0.6666666666666666, '27': 0.7, '28': 0.6625, '31': 0.6708333333333333, '34': 0.6166666666666667, '35': 0.7333333333333333}
所有被试的平均准确率: 67.2%
所有被试准确率（无名称）: 65.4,66.7,70.0,66.2,67.1,61.7,73.3


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0_20_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 20
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 87.9%
subject 24 的准确率: 85.8%
subject 27 的准确率: 75.4%
subject 28 的准确率: 92.5%
subject 31 的准确率: 82.5%
subject 34 的准确率: 82.5%
subject 35 的准确率: 85.8%
所有被试的准确率： {'22': 0.8791666666666667, '24': 0.8583333333333333, '27': 0.7541666666666667, '28': 0.925, '31': 0.825, '34': 0.825, '35': 0.8583333333333333}
所有被试的平均准确率: 84.6%
所有被试准确率（无名称）: 87.9,85.8,75.4,92.5,82.5,82.5,85.8


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0_30_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 30
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 85.0%
subject 24 的准确率: 81.7%
subject 27 的准确率: 71.7%
subject 28 的准确率: 87.5%
subject 31 的准确率: 78.3%
subject 34 的准确率: 76.2%
subject 35 的准确率: 85.4%
所有被试的准确率： {'22': 0.85, '24': 0.8166666666666667, '27': 0.7166666666666667, '28': 0.875, '31': 0.7833333333333333, '34': 0.7625, '35': 0.8541666666666666}
所有被试的平均准确率: 80.8%
所有被试准确率（无名称）: 85.0,81.7,71.7,87.5,78.3,76.2,85.4


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0_40_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 40
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 66.7%
subject 24 的准确率: 64.6%
subject 27 的准确率: 64.2%
subject 28 的准确率: 70.0%
subject 31 的准确率: 70.8%
subject 34 的准确率: 62.1%
subject 35 的准确率: 74.6%
所有被试的准确率： {'22': 0.6666666666666666, '24': 0.6458333333333334, '27': 0.6416666666666667, '28': 0.7, '31': 0.7083333333333334, '34': 0.6208333333333333, '35': 0.7458333333333333}
所有被试的平均准确率: 67.6%
所有被试准确率（无名称）: 66.7,64.6,64.2,70.0,70.8,62.1,74.6


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0_50_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 50
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 66.2%
subject 24 的准确率: 63.3%
subject 27 的准确率: 62.5%
subject 28 的准确率: 61.3%
subject 31 的准确率: 70.8%
subject 34 的准确率: 60.4%
subject 35 的准确率: 72.1%
所有被试的准确率： {'22': 0.6625, '24': 0.6333333333333333, '27': 0.625, '28': 0.6125, '31': 0.7083333333333334, '34': 0.6041666666666666, '35': 0.7208333333333333}
所有被试的平均准确率: 65.2%
所有被试准确率（无名称）: 66.2,63.3,62.5,61.3,70.8,60.4,72.1


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0_60_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 60
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 65.4%
subject 24 的准确率: 66.7%
subject 27 的准确率: 70.0%
subject 28 的准确率: 66.2%
subject 31 的准确率: 67.1%
subject 34 的准确率: 61.7%
subject 35 的准确率: 73.3%
所有被试的准确率： {'22': 0.6541666666666667, '24': 0.6666666666666666, '27': 0.7, '28': 0.6625, '31': 0.6708333333333333, '34': 0.6166666666666667, '35': 0.7333333333333333}
所有被试的平均准确率: 67.2%
所有被试准确率（无名称）: 65.4,66.7,70.0,66.2,67.1,61.7,73.3


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0_70_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon =70
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 62.5%
subject 24 的准确率: 68.8%
subject 27 的准确率: 62.5%
subject 28 的准确率: 60.4%
subject 31 的准确率: 63.7%
subject 34 的准确率: 55.8%
subject 35 的准确率: 72.5%
所有被试的准确率： {'22': 0.625, '24': 0.6875, '27': 0.625, '28': 0.6041666666666666, '31': 0.6375, '34': 0.5583333333333333, '35': 0.725}
所有被试的平均准确率: 63.7%
所有被试准确率（无名称）: 62.5,68.8,62.5,60.4,63.7,55.8,72.5


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0_80_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 80
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 62.5%
subject 24 的准确率: 64.6%
subject 27 的准确率: 62.5%
subject 28 的准确率: 62.5%
subject 31 的准确率: 63.3%
subject 34 的准确率: 59.2%
subject 35 的准确率: 73.8%
所有被试的准确率： {'22': 0.625, '24': 0.6458333333333334, '27': 0.625, '28': 0.625, '31': 0.6333333333333333, '34': 0.5916666666666667, '35': 0.7375}
所有被试的平均准确率: 64.1%
所有被试准确率（无名称）: 62.5,64.6,62.5,62.5,63.3,59.2,73.8


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0_90_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 90
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 66.2%
subject 24 的准确率: 64.6%
subject 27 的准确率: 60.0%
subject 28 的准确率: 67.5%
subject 31 的准确率: 62.1%
subject 34 的准确率: 58.3%
subject 35 的准确率: 74.6%
所有被试的准确率： {'22': 0.6625, '24': 0.6458333333333334, '27': 0.6, '28': 0.675, '31': 0.6208333333333333, '34': 0.5833333333333334, '35': 0.7458333333333333}
所有被试的平均准确率: 64.8%
所有被试准确率（无名称）: 66.2,64.6,60.0,67.5,62.1,58.3,74.6


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0_100_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 0
epsilon = 100
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 62.9%
subject 24 的准确率: 64.2%
subject 27 的准确率: 70.0%
subject 28 的准确率: 67.5%
subject 31 的准确率: 70.4%
subject 34 的准确率: 60.4%
subject 35 的准确率: 73.8%
所有被试的准确率： {'22': 0.6291666666666667, '24': 0.6416666666666667, '27': 0.7, '28': 0.675, '31': 0.7041666666666667, '34': 0.6041666666666666, '35': 0.7375}
所有被试的平均准确率: 67.0%
所有被试准确率（无名称）: 62.9,64.2,70.0,67.5,70.4,60.4,73.8
